In [1]:
# !pip install --user tensorflow

import gensim
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
df= pd.read_csv("sentimentsdata_V3.csv")
df = df.rename(columns={'News Article': 'Article'})

In [3]:
columns_to_drop = ['Unnamed: 0', 'Unnamed: 0.1']
df = df.drop(columns=columns_to_drop)
df['Remark'] = df['Remark'].replace({'sadness': 'Sadness'})

In [4]:
#remove the articles which have neither panzer nor leopard in them 
df=df.drop(118)
df=df.drop(239)
df=df.drop(583)

In [5]:
df

,Article,News Date,Sentiment,Remark
0,Während die Ukraine weiter auf Kampfpanzer aus...,"January 23, 2023",3,Anger
1,"Es kommt selten vor, dass ein General öffentli...","July 30, 2023",3,Anger
2,VON MARKUS DECKER UND SVEN CHRISTIAN SCHULZ (R...,"December 1, 2022",3,Anger
3,Sie können es nicht richtig machen Heft 41/202...,"October 14, 2022",5,Disgust
4,Ein Ausweg aus dem Wolkenkuckucksheim Immer wi...,"February 2, 2023",5,Disgust
...,...,...,...,...
751,Ramstein. Auch bei dem hochrangig besetzten Tr...,"January 21, 2023",7,Trust
752,Ein Leser teilt seine Ansichten zum Krieg in d...,"January 27, 2023",7,Trust
753,Wir treffen den ukrainischen Botschafter in se...,"June 17, 2023",7,Trust
754,Seit einem Jahr greift Russland die Ukraine an...,"February 19, 2023",6,Surprise


In [6]:
Articles = df.Article.apply(gensim.utils.simple_preprocess)

In [8]:
Articles

0      [während, die, ukraine, weiter, auf, kampfpanz...
1      [es, kommt, selten, vor, dass, ein, general, ö...
2      [von, markus, decker, und, sven, christian, sc...
3      [sie, können, es, nicht, richtig, machen, heft...
4      [ein, ausweg, aus, dem, immer, wieder, verwund...
                             ...                        
751    [ramstein, auch, bei, dem, hochrangig, besetzt...
752    [ein, leser, teilt, seine, ansichten, zum, kri...
753    [wir, treffen, den, ukrainischen, botschafter,...
754    [seit, einem, jahr, greift, russland, die, ukr...
755    [aus, madrid, und, la, coruña, anna, lehmann, ...
Name: Article, Length: 753, dtype: object

In [7]:
def extract_from_text(texts, delta=50):
    sections = []
    for words in texts:
        parts = ""
        keyword_idx = []
        for count, i in enumerate(words):
            if ("panzer" in i) or ("leopard" in i):
                keyword_idx.append(count)

        intervals = []
        for i in keyword_idx:
            intervals.append([i - delta, i + delta])
        if len(intervals) > 0:
            merged_intervals = [intervals[0]]
            for i in range(1, len(intervals)):
                current_interval = intervals[i]
                last_merged = merged_intervals[-1]
                if current_interval[0] <= last_merged[1]:
                    merged_intervals[-1] = [last_merged[0], max(last_merged[1], current_interval[1])]
                else:
                    merged_intervals.append(current_interval)

            for i in merged_intervals:
                part = []
                if i[0] < 0:
                    i[0] = 0
                for j in range(i[0], i[1]+1):
                    if j >= len(words):
                        break
                    part.append(words[j])

                parts += " | " + " ".join(part)

            sections.append(parts)

    return sections

extracted = extract_from_text(Articles)

In [8]:
len(extracted)

753

In [9]:
extracted[1]

' | und fördert sie bedingt abwehrbereit die bundeswehr ist eine dafür sorgen nicht nur mehr als sondern auch zehntausende soldaten deren tätigkeiten eher denen in grossen behörden gleichen denn denen einer truppe die den ernstfall trainiert kampfkraft erzeugen diese nicht sie sitzen im kriegsfall nicht in gräben und stellungen kampf und schützenpanzern schiffen und booten kosten aber viel geld die bundeswehr gibt inzwischen die hälfte ihres budgets für personal aus doch eine armee mit zu vielen häuptlingen für zu wenig indianer ist mit der neuen konfrontation in europa überfordert der krieg in der ukraine zeigt worauf es in einem konflikt an der | papier vor in dem es hiess dass russland pro tag zwischen und verbrauche was die bundeswehr brauchte wäre ein masterplan munition um die depots aufzufüllen doch auch anderthalb jahre nach dem russischen überfall dümpelt die beschaffung vor sich hin anfang juli genehmigte der des bundestags den kauf von artillerie und panzermunition im umfang

In [10]:
Articles_processed = [gensim.utils.simple_preprocess(section) for section in extracted]

In [11]:
len(Articles_processed)

753

In [12]:
model= gensim.models.Word2Vec(
    window=10,
    min_count=1,
    workers=4
    )

In [13]:
model.epochs=15
model.epochs

15

In [14]:
model.build_vocab(Articles_processed, progress_per=1000)

In [15]:
model.train(Articles_processed, total_examples=model.corpus_count, epochs=model.epochs)

(2557508, 3419700)

In [15]:
model.corpus_count

753

In [16]:
model.wv.most_similar("leopard")

[('export', 0.7070510983467102),
 ('hergestellte', 0.7029938101768494),
 ('genehmigung', 0.7026856541633606),
 ('spanien', 0.696653425693512),
 ('polen', 0.6920331716537476),
 ('portugal', 0.6817083358764648),
 ('schweden', 0.6801103949546814),
 ('gestattet', 0.675040066242218),
 ('typs', 0.6744887232780457),
 ('zugesagten', 0.6666199564933777)]

In [17]:
model.wv.similarity(w1="schlecht", w2= "gut")

0.78172094

In [18]:
def tokens_to_embeddings(tokens, model):
    embeddings = []
    for token in tokens:
        if token in model.wv:
            embeddings.append(model.wv[token])
    return embeddings

X = [tokens_to_embeddings(tokens, model) for tokens in Articles_processed]

In [19]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len=300
X_padded = pad_sequences(X, maxlen=max_len, dtype='float32', padding='post', truncating='post')


In [20]:
from sklearn.model_selection import train_test_split
import numpy as np

x_train, x_test, y_train, y_test = train_test_split(X_padded, df['Sentiment'].values, test_size=0.2)


In [21]:
from tensorflow.keras.utils import to_categorical

num_classes = 8

y_train_encoded = to_categorical(y_train, num_classes=num_classes)
y_test_encoded=to_categorical(y_test, num_classes=num_classes)
y_test_encoded.shape

(151, 8)

In [22]:
from keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization

model_lstm= Sequential()
model_lstm.add(LSTM(256, return_sequences=True))
model_lstm.add(Dropout(0.3))
model_lstm.add(LSTM(256, return_sequences=False))
model_lstm.add(Dense(8, activation='softmax'))  

In [23]:
model_lstm.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Use categorical_crossentropy for multi-class
              metrics=['accuracy'])

In [24]:
model_lstm.fit(x_train, y_train_encoded, validation_data=(x_test,y_test_encoded), epochs=10, batch_size=32)

Epoch 1/10
19/19 [==============================] - 51s 2s/step - loss: 1.9926 - accuracy: 0.2193 - val_loss: 1.9027 - val_accuracy: 0.2781
Epoch 2/10
19/19 [==============================] - 42s 2s/step - loss: 1.8926 - accuracy: 0.2492 - val_loss: 1.8657 - val_accuracy: 0.2649
Epoch 3/10
19/19 [==============================] - 46s 2s/step - loss: 1.8428 - accuracy: 0.2641 - val_loss: 1.9033 - val_accuracy: 0.2517
Epoch 4/10
19/19 [==============================] - 47s 3s/step - loss: 1.7713 - accuracy: 0.2924 - val_loss: 1.9445 - val_accuracy: 0.2517
Epoch 5/10
19/19 [==============================] - 46s 2s/step - loss: 1.6876 - accuracy: 0.3439 - val_loss: 2.0343 - val_accuracy: 0.2384
Epoch 6/10
19/19 [==============================] - 47s 2s/step - loss: 1.6080 - accuracy: 0.3588 - val_loss: 2.0502 - val_accuracy: 0.2715
Epoch 7/10
19/19 [==============================] - 44s 2s/step - loss: 1.5085 - accuracy: 0.3771 - val_loss: 2.1555 - val_accuracy: 0.2185
Epoch 8/10
19/19 [==